<a href="https://colab.research.google.com/github/elikaaghaei/Rahnema_college/blob/main/%22ML%20notebooks%22/MNIST_TF_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import tensorflow as tf
from tensorflow.keras import Model, layers
import numpy as np
import tensorflow.keras as keras

# Initialization

In [36]:
num_classes = 10  #total classes (0-9 digits).

#Training Parameters
learning_rate = 0.001
trainig_steps = 200
batch_size = 128
display_step = 10

#Network Parameters
conv1_filters = 32 #number of filters for the first conv layer
conv2_filters = 64 #number of filters for the second conv layer
fc1_units = 1024 # umber of neurons for the fully connected layer

### Prepare Dataset

In [37]:
from keras.datasets import mnist
(x_train, y_train), (x_test,y_test) = mnist.load_data()

x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)

In [38]:
x_train , x_test = x_train/255.0 , x_test /255.0

In [39]:
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))

In [40]:
train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

## Create TF model

In [41]:
class ConvNet(Model):
  #set layers
  def __init__(self):
    super(ConvNet, self).__init__()

    self.conv1 = layers.Conv2D(32, kernel_size=5, activation=tf.nn.relu)
    self.maxpool1 = layers.MaxPool2D(2, strides=2)
    self.conv2 = layers.Conv2D(64 , kernel_size=3, activation = tf.nn.relu)
    self.maxpool2 = layers.MaxPool2D(2, strides =2)
    self.flatten = layers.Flatten()
    self.fc1 = layers.Dense(1024)

    #self.fc2 = layers.Dense(2048)
    #self.dropout2 = layers.Dropout(rate =0.5)

    self.dropout = layers.Dropout(rate =0.2)
    self.out = layers.Dense(num_classes)

  def call(self, x, is_training = False):
    x = tf.reshape (x, [-1,28,28,1]) #?????
    x = self.conv1(x)
    x = self.maxpool1(x)
    x = self.conv2(x)
    x = self.maxpool2(x)
    x = self.flatten(x)
    x = self.fc1(x)

    x = self.dropout(x, training = is_training)
    #x = self.fc2(x)
    #x = self.dropout2(x, training = is_training)
    x = self.out(x)

    if not is_training:
      x = tf.nn.softmax(x)

    return x

### Create an object of that

In [42]:
conv_net = ConvNet()

In [43]:
def cross_entropy_loss(x,y):
  y = tf.cast(y, tf.int64)

  loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=x)
  return tf.reduce_mean(loss)

def accuracy(y_pred, y_true):
  correct_prediction = tf.equal(tf.argmax(y_pred,1), tf.cast(y_true, tf.int64))
  return tf.reduce_mean(tf.cast(correct_prediction, tf.float32), axis=-1)

optimizer = tf.optimizers.Adam(learning_rate = learning_rate)

### Optimization Process

In [44]:
@tf.function
def run_optimization(x,y):
  with tf.GradientTape() as g:
    pred = conv_net(x, is_training = True)
    loss = cross_entropy_loss(pred,y)

  trainable_variables = conv_net.trainable_variables

  gradient = g.gradient(loss, trainable_variables)

  optimizer.apply_gradients(zip(gradient, trainable_variables))

### Run training for a given data

In [45]:
for step, (batch_x, batch_y) in enumerate(train_data.take(trainig_steps),1):
  run_optimization(batch_x, batch_y)

  if step % display_step ==0:
    pred = conv_net(batch_x)
    loss = cross_entropy_loss(pred, batch_y)
    acc = accuracy(pred, batch_y)
    print("steps: %i, loss: %f, accuracy: %f" %(step,loss,acc))

steps: 10, loss: 1.820090, accuracy: 0.820312
steps: 20, loss: 1.624205, accuracy: 0.867188
steps: 30, loss: 1.592002, accuracy: 0.906250
steps: 40, loss: 1.558525, accuracy: 0.945312
steps: 50, loss: 1.514492, accuracy: 0.976562
steps: 60, loss: 1.548636, accuracy: 0.960938
steps: 70, loss: 1.511889, accuracy: 0.960938
steps: 80, loss: 1.530424, accuracy: 0.953125
steps: 90, loss: 1.492814, accuracy: 1.000000
steps: 100, loss: 1.511071, accuracy: 0.984375
steps: 110, loss: 1.506704, accuracy: 0.984375
steps: 120, loss: 1.491405, accuracy: 0.992188
steps: 130, loss: 1.500378, accuracy: 0.968750
steps: 140, loss: 1.500151, accuracy: 0.960938
steps: 150, loss: 1.525207, accuracy: 0.953125
steps: 160, loss: 1.492911, accuracy: 0.976562
steps: 170, loss: 1.515489, accuracy: 0.960938
steps: 180, loss: 1.488935, accuracy: 1.000000
steps: 190, loss: 1.497555, accuracy: 0.968750
steps: 200, loss: 1.481817, accuracy: 0.992188


In [46]:
pred = conv_net(x_test)

print('Test Accuracy: %f' % accuracy(pred,y_test))

Test Accuracy: 0.980600


- we can use it via keras

- We can plot it